## CRITICAL STEPS BEFORE PROCESSING ONBOARDING CUSTOMER USAGE DATA WITH THIS SCRIPT: 
### Identify the below columns within each data artifact
### Update header naming conventions to match the below spelling and capitalization EXACTLY

#### USAGE DATA FILE INPUTS NEEDED:
    BUSN Unit
    HELPERKEY
    ORDER DATE
    ORDER UOM
    ORDER QTY
    Cust UOM (optional)
    Cust UOM Conv Factor (optional)

#### PXR SCORECARD INPUTS NEEDED:
    ADD/CHANGE DATE (ATS)
    READY FOR RELEASE TO ATS/PURCHASING
    SCORECARD CATEGORY
    GO FORWARD SKU TO USE

#### DC/ACCOUNT METADATA FILE INPUTS NEEDED:
    BUSN Unit
    LOC
    OM ACCOUNT

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import math
import oracledb
import os
import re
import pandas as pd

#### Enter necessary metadata here ####
project_folder = '2024-12-09 University Hospital ABC'
scorecard_date_prev = '2024-10-24'          # Enter date of the last scorecard used in ATS processing
scorecard_date = '2024-11-06'               # Enter date of scorecard information needed to be processed now
usageformat = 'PO'                          # Either 'PO' or 'monthly'
numberofmonths = 12                          # enter the number of months, if data is in a monthly format
ATS_workflow_type = 'update'                  # Either 'initial' or 'update'
ATS_workflow_type_previousfile = 'update'     # Either 'initial' or 'update'

# Capture the customer name using regex
customer_name = re.search(r'\d{4}-\d{2}-\d{2}\s(.+)', project_folder).group(1)

# Base directory path
base_dir = 'C:/your/filepath/here/'
projectinputs_folder = os.path.join(base_dir, project_folder, 'Inputs')
projectoutputs_folder = os.path.join(base_dir, project_folder, 'Outputs')

# Construct the file paths
if ATS_workflow_type == 'initial':
    data_usage_filepath = os.path.join(projectinputs_folder, f'{customer_name}_Usage_Initial.xlsx')
elif ATS_workflow_type == 'update':
    data_usage_filepath = os.path.join(projectoutputs_folder, f'{customer_name}_ATSFile_{ATS_workflow_type_previousfile}_{scorecard_date_prev}.xlsx')

data_scorecard_filepath = os.path.join(projectinputs_folder, f'{customer_name}_Scorecard_{scorecard_date}.xlsb')
df_metadata_filepath = os.path.join(projectinputs_folder, f'{customer_name}_DC_Acct_Data.xlsx')

# Function to read Excel files with error handling
def read_excel_file(filepath, **kwargs):
    try:
        return pd.read_excel(filepath, **kwargs)
    except FileNotFoundError:
        print(f"Error: File not found - {filepath}")
        return None
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

# Read in necessary data artifacts
data_usage = read_excel_file(data_usage_filepath, sheet_name='Raw Usage', dtype={'MMIS ID': str})
data_scorecard = read_excel_file(data_scorecard_filepath, sheet_name='Raw Data', engine='pyxlsb')
df_metadata = read_excel_file(df_metadata_filepath)

In [2]:
### Clean scorecard data

# Make a copy dataframe of raw PXR scorecard data
df_scorecard = data_scorecard.copy()

# Rename the primary SCORECARD CATEGORY column
df_scorecard.rename(
    columns={col: "SCORECARD CATEGORY" for col in df_scorecard.filter(like="SCORECARD CATEGORY").columns if "PREVIOUS" not in col},
    inplace=True
)

# Define new column names with the scorecard date prefix
new_column_names = {
    'ADD/CHANGE DATE (ATS)': f'{scorecard_date} ADD/CHANGE DATE (ATS)',
    'READY FOR RELEASE TO ATS/PURCHASING': f'{scorecard_date} READY RELEASE ATS',
    'SCORECARD CATEGORY': f'{scorecard_date} SCORECARD CATEGORY',
    'GO FORWARD SKU TO USE': f'{scorecard_date} OM SKU',   
}

# Apply new column names
df_scorecard.rename(columns=new_column_names, inplace=True)

# Filter for only necessary scorecard columns
df_scorecard = df_scorecard[['HELPERKEY', f'{scorecard_date} READY RELEASE ATS',
                             f'{scorecard_date} ADD/CHANGE DATE (ATS)',
                             f'{scorecard_date} SCORECARD CATEGORY',
                             f'{scorecard_date} OM SKU']]

# Clean and change datatypes of given columns. Make everything upper case, trim leading and trailing spaces around OM SKUs
df_scorecard['HELPERKEY'] = df_scorecard['HELPERKEY'].astype('str').str.upper()
df_scorecard[f'{scorecard_date} READY RELEASE ATS'] = df_scorecard[f'{scorecard_date} READY RELEASE ATS'].str.upper()
df_scorecard[f'{scorecard_date} OM SKU'] = df_scorecard[f'{scorecard_date} OM SKU'].str.strip()

# Sort dataframe to put all 'YES' at the top
df_scorecard.sort_values(by=f'{scorecard_date} READY RELEASE ATS', ascending=False, inplace=True)

# Drop duplicate HELPERKEYs from scorecard. If this step is omitted, usage data will be duplicated due to data joins later on in this script
df_scorecard.drop_duplicates(subset='HELPERKEY', keep='first', inplace=True)

# Create dataframe with unique OM SKUs that are 'YES' for ready release
df_unique_ready_release_om_skus = (
    df_scorecard[df_scorecard[f'{scorecard_date} READY RELEASE ATS'] == 'YES']
    .drop_duplicates(subset=[f'{scorecard_date} OM SKU'])
    [[f'{scorecard_date} OM SKU']]
)

In [3]:
# Function to chunk a dataframe into chunks of specified size
def chunk_list(dataframe, chunk_size=1000):
    return [dataframe[i:i + chunk_size] for i in range(0, len(dataframe), chunk_size)]

# Function to replace specific values in a column
def replace_values(col):
    return col.replace(['u', 0], np.nan)

# Establish the database connection using a context manager to ensure it closes properly
dsn_tns = oracledb.makedsn('hostnamehere', '1234', service_name='DATABASE_NAME_HERE')

with oracledb.connect(user='usernamehere', password='pswdhere', dsn=dsn_tns) as conn:
    cur = conn.cursor()

    # Create empty lists to hold the SQL query results for later concatenation
    results_by = []
    results_edw = []

    # Break up unique_skus into chunks of 1000
    chunks = chunk_list(df_unique_ready_release_om_skus, chunk_size=1000)

    # Create a list of unique OM DCs
    unique_dcs = ', '.join([f"'{dc}'" for dc in df_metadata['LOC'].unique()])

    ### Run SQL query for BY data in chunks of 1000 OM SKUs
    for chunk in chunks:
        unique_skus_str = ', '.join([f"'{sku}'" for sku in chunk[f'{scorecard_date} OM SKU']])

        query_by = f"""
        SELECT
            s.loc,
            s.item AS "{scorecard_date} OM SKU",
            s.UDC_4WEEKFCST AS "DC 4wk fcst",
            s.planleadtime/1440 AS "Days Lead Time",
            s.iflag
        FROM SCPDBINSTANCE.SKU s
        WHERE s.loc IN ({unique_dcs})
        AND s.item IN ({unique_skus_str})
        ORDER BY s.item
        """

        try:
            cur.execute(query_by)
            rows = cur.fetchall()
            column_names = [column[0] for column in cur.description]
            results_by.append(pd.DataFrame(rows, columns=column_names))
        except Exception as e:
            print(f"Error during BY query execution: {e}")

    # Concatenate all chunks into one DataFrame
    df_by = pd.concat(results_by, ignore_index=True)

    ### Run SQL query for EDW data in chunks of 1000 OM SKUs
    for chunk in chunks:
        unique_skus_str = ', '.join([f"'{sku}'" for sku in chunk[f'{scorecard_date} OM SKU']])

        query_edw = f"""
        SELECT
            pd.ID_SKU AS "{scorecard_date} OM SKU", pd.NM_SKU AS "OM Item Description",
            (pd.AT_CORP_BASE_COST * pd.RT_PURCHASE_UOM_CONVERSION) AS "$ PUOM Unit Cost",
            pd.CD_BASE_UOM AS "Base UOM", pd.DS_BASE_UOM_FACTOR AS "Base UOM Factor",
            pd.CD_PURCHASE_UOM AS "Purchase UOM", pd.RT_PURCHASE_UOM_CONVERSION AS "Purchase UOM Factor",
            pd.CD_ALTERNATE_UOM_1 AS "Alt UOM 1", pd.RT_ALTERNATE_UOM_CONV_1 AS "Alt UOM 1 Factor",
            pd.CD_ALTERNATE_UOM_2 AS "Alt UOM 2", pd.RT_ALTERNATE_UOM_CONV_2 AS "Alt UOM 2 Factor",
            pd.CD_ALTERNATE_UOM_3 AS "Alt UOM 3", pd.RT_ALTERNATE_UOM_CONV_3 AS "Alt UOM 3 Factor",
            pd.CD_ALTERNATE_UOM_4 AS "Alt UOM 4", pd.RT_ALTERNATE_UOM_CONV_4 AS "Alt UOM 4 Factor",
            pd.CD_ALTERNATE_UOM_5 AS "Alt UOM 5", pd.RT_ALTERNATE_UOM_CONV_5 AS "Alt UOM 5 Factor",
            pd.CD_ALTERNATE_UOM_6 AS "Alt UOM 6", pd.RT_ALTERNATE_UOM_CONV_6 AS "Alt UOM 6 Factor"
        FROM DBINSTANCE.PRODUCT_DIM pd
        WHERE pd.ID_SKU IN ({unique_skus_str})
        AND pd.FG_CURRENT = 'Y'
        AND pd.FG_ACTIVE_SKU = 'ACTIVE'
        """

        try:
            cur.execute(query_edw)
            rows = cur.fetchall()
            column_names = [column[0] for column in cur.description]
            results_edw.append(pd.DataFrame(rows, columns=column_names))
        except Exception as e:
            print(f"Error during EDW query execution: {e}")

    # Concatenate all chunks into one DataFrame
    df_edw = pd.concat(results_edw, ignore_index=True)

# Remove duplicate OM SKUs from the EDW dataframe
df_edw = df_edw.drop_duplicates(subset=f'{scorecard_date} OM SKU', keep='first')

# Remove 'na' data from scorecard UOM Conversion Factor data and replace 'u' and 0 with NULL
alt_cols = [col for col in df_edw.columns if col.startswith('Alt')]
df_edw[alt_cols] = df_edw[alt_cols].apply(replace_values)

# Format any columns that end with the word "Factor" into a float
factor_cols = [col for col in df_edw.columns if col.endswith('Factor')]
df_edw[factor_cols] = df_edw[factor_cols].astype(float)

# Check for OM SKUs not found in the EDW UOM conversion data pull
missing_records = df_unique_ready_release_om_skus[~df_unique_ready_release_om_skus[f'{scorecard_date} OM SKU'].isin(df_edw[f'{scorecard_date} OM SKU'])]
print("OM SKUs not found in PRODUCT_DIM table: ", missing_records)

# Output the number of SKUs from the data pull
print("# of SKUs from data pull: ", len(df_edw))

OM SKUs not found in PRODUCT_DIM table:  Empty DataFrame
Columns: [2024-11-06 OM SKU]
Index: []
# of SKUs from data pull:  1259


In [4]:
### Validate PXR scorecard & usage data

# Make a copy of raw usage data
df_usage = data_usage.copy()

# Change datatype of HELPERKEY and make it uppercase
df_usage['HELPERKEY'] = df_usage['HELPERKEY'].astype(str).str.upper()

# Check and convert 'ORDER DATE' to datetime format
df_usage['ORDER DATE'] = pd.to_datetime(df_usage['ORDER DATE'])
df_usage['ORDER YEAR-Mth'] = df_usage['ORDER DATE'].dt.strftime('%Y-%m')

# Group by 'HELPERKEY' and count unique 'OM SKU' values for one-to-many relationships
df_helperkey_to_multiple_om_skus = (
    df_scorecard.groupby('HELPERKEY')[f'{scorecard_date} OM SKU'].nunique()
    .loc[lambda x: x > 1].reset_index()
)

# Rows where READY RELEASE ATS == 'YES' but no OM SKU
df_ready_release_yes_no_om_sku = df_scorecard[
    (df_scorecard[f'{scorecard_date} READY RELEASE ATS'] == 'YES') &
    (df_scorecard[f'{scorecard_date} OM SKU'].isnull() | (df_scorecard[f'{scorecard_date} OM SKU'] == 0))
]

# Rows where READY RELEASE ATS == 'no' but SCORECARD CATEGORY contains 'EXACT MATCH' or 'APPROVED'
df_no_ready_release_exact_match_approved = df_scorecard[
    (df_scorecard[f'{scorecard_date} READY RELEASE ATS'] == 'no') &
    df_scorecard[f'{scorecard_date} SCORECARD CATEGORY'].str.contains('EXACT MATCH|APPROVED', case=False, na=False)
]

# Rows where READY RELEASE ATS == 'YES' but NOT an exact match or approved sub
df_ready_release_yes_not_exact_match_approved = df_scorecard[
    (df_scorecard[f'{scorecard_date} READY RELEASE ATS'] == 'YES') &
    ~df_scorecard[f'{scorecard_date} SCORECARD CATEGORY'].str.contains('EXACT MATCH|APPROVED', case=False, na=False)
]

# Look for records in usage data that did not match to scorecard data
df_nomatchkey = pd.merge(df_usage, df_scorecard, on='HELPERKEY', how='outer', indicator=True)
df_nomatchkey = df_nomatchkey[df_nomatchkey['_merge'] == 'left_only']

# Display validation results
print(f'There are {len(df_helperkey_to_multiple_om_skus)} rows where there are several OM SKUs to one single HELPERKEY.')
print(f'There are {len(df_ready_release_yes_no_om_sku)} PXR records that are marked YES ready release for ATS and do not have an OM SKU attached.')
print(f'There are {len(df_no_ready_release_exact_match_approved)} records that are an exact match or approved sub, but are marked NO ready release for ATS.')
print(f'There are {len(df_ready_release_yes_not_exact_match_approved)} records that are NOT an exact match or approved sub, but are marked YES ready release for ATS.')
print(f'There are {len(df_nomatchkey)} records that did not match to scorecard data.')

There are 0 rows where there are several OM SKUs to one single HELPERKEY.
There are 0 PXR records that are marked YES ready release for ATS and do not have an OM SKU attached.
There are 0 records that are an exact match or approved sub, but are marked NO ready release for ATS.
There are 13 records that are NOT an exact match or approved sub, but are marked YES ready release for ATS.
There are 0 records that did not match to scorecard data.


In [5]:
df_usage1 = df_usage.copy()

# Merge with DC and Account data
if ATS_workflow_type == 'initial':
    df_usage1 = df_usage1.merge(df_metadata, on='BUSN Unit', how='left')

# Merge scorecard data with EDW data
df_scorecard = df_scorecard.merge(df_edw, on=f'{scorecard_date} OM SKU', how='left')

if ATS_workflow_type == 'initial':
    # Capture column headers from raw usage data
    rawusagecolumns = df_usage1.columns

    # Merge scorecard data with usage data
    df_usagewithscorecard = df_usage1.merge(df_scorecard, on='HELPERKEY', how='left')

elif ATS_workflow_type == 'update':
    # Remove previous weeks' metadata, then capture column headers from raw usage data
    rawusagecolumns = list(df_usage1.columns[:-20])  # Explicit column list for clarity

    # Bring in new UOM/EDW data
    df_usagewithscorecard = pd.merge(df_usage1[rawusagecolumns], df_scorecard, on='HELPERKEY', how='left')

# Convert ORDER QTYs @ OM SKU's Purchase UOM
def calculate_converted_qty(df):
    num_alt_uoms = 6  # Change this to match the number of Alt UOMs available
    uom_columns = ['Purchase UOM', 'Base UOM'] + [f'Alt UOM {i}' for i in range(1, num_alt_uoms + 1)]
    factors = ['Purchase UOM Factor', 'Base UOM Factor'] + [f'Alt UOM {i} Factor' for i in range(1, num_alt_uoms + 1)]

    # Initialize ORDER QTY CONVERTED column
    df['ORDER QTY CONVERTED'] = 0.0

    for uom, factor in zip(uom_columns, factors):
        if uom in df.columns and factor in df.columns:
            mask = df['ORDER UOM'] == df[uom]
            if uom == 'Purchase UOM':
                df.loc[mask, 'ORDER QTY CONVERTED'] = df.loc[mask, 'ORDER QTY']
            elif uom == 'Base UOM':
                df.loc[mask, 'ORDER QTY CONVERTED'] = df.loc[mask, 'ORDER QTY'] / df.loc[mask, 'Purchase UOM Factor']
            else:
                df.loc[mask, 'ORDER QTY CONVERTED'] = (df.loc[mask, 'ORDER QTY'] * df.loc[mask, factor] / df.loc[mask, 'Purchase UOM Factor'])

    # Create conversion error notes
    df['Conversion Error Note'] = df['ORDER QTY CONVERTED'].isna().replace({True: "NO UOM Match", False: ""})
    return df

# Apply conversion function to the usage data
df_usage_converted = calculate_converted_qty(df_usagewithscorecard.copy())

# Create dataframe to highlight UOM conversion issues
df_conversionerror = (
    df_usage_converted.loc[df_usage_converted['Conversion Error Note'].eq('NO UOM Match'), 
                           ['HELPERKEY', 'MMIS ID', f'{scorecard_date} OM SKU', 'ORDER UOM']]
    .drop_duplicates()
)

# Create a function that categorizes each row by what changes happened between current and previous week's scorecards and input the category into a new column 'PXR Update Category'
def update_pxr_category(df, scorecard_date_prev, scorecard_date):
    # Initialize 'PXR Update Category' column with empty strings
    df['PXR Update Category'] = ''

    # Condition 1: 'YES TO NO: ATS Removes'
    condition1 = (df[f'{scorecard_date_prev} READY RELEASE ATS'] == 'YES') & (df[f'{scorecard_date} READY RELEASE ATS'] == 'NO')
    df.loc[condition1, 'PXR Update Category'] = 'YES TO NO: ATS Removes'

    # Condition 2: 'SKU CHANGE'
    condition2 = (
        (df[f'{scorecard_date_prev} OM SKU'] != df[f'{scorecard_date} OM SKU']) &
        df[f'{scorecard_date_prev} OM SKU'].notnull() &
        (df[f'{scorecard_date_prev} READY RELEASE ATS'] == 'YES') & 
        (df[f'{scorecard_date} READY RELEASE ATS'] == 'YES')
    )
    df.loc[condition2, 'PXR Update Category'] = 'SKU CHANGE'

    # Condition 3: 'NO TO YES: NEW ADDS'
    condition3 = (df[f'{scorecard_date_prev} READY RELEASE ATS'] == 'NO') & (df[f'{scorecard_date} READY RELEASE ATS'] == 'YES')
    df.loc[condition3, 'PXR Update Category'] = 'NO TO YES: NEW ADDS'

    return df

# Merge new scorecard updates into initial usage data w/ PXR scorecard data from Initial ATS File
if ATS_workflow_type == 'initial':
    # Filter for Y ready release items, filter out items with 0 usage or UOM conversion issues
    df_usage_converted_YesATS = df_usage_converted[
        (df_usage_converted['Conversion Error Note'] == '') &
        (df_usage_converted[f'{scorecard_date} READY RELEASE ATS'] == 'YES')
    ]

elif ATS_workflow_type == 'update':
    # Ensure update_pxr_category function is modifying the dataframe as expected
    df_usage_converted = update_pxr_category(df_usage_converted, scorecard_date_prev, scorecard_date)

    # Filter for Y ready release items with PXR Update Category conditions
    df_usage_converted_YesATS = df_usage_converted[
        (df_usage_converted['Conversion Error Note'] == '') &
        (df_usage_converted['PXR Update Category'].isin(['NO TO YES: NEW ADDS', 'SKU CHANGE']))
    ]

    # Create pivot of removes that may need to be done
    df_updates_removes = (df_usage_converted[df_usage_converted['PXR Update Category'].isin(['YES TO NO: ATS Removes', 'SKU CHANGE'])]
                                   .groupby(['LOC', 'OM ACCOUNT', f'{scorecard_date_prev} OM SKU', f'{scorecard_date} OM SKU', f'{scorecard_date} SCORECARD CATEGORY', f'{scorecard_date} ADD/CHANGE DATE (ATS)'])
                                    .size()
                                    .reset_index(name='Count'))


In [6]:
# Function to pivot PO data into a monthly format
def create_pivot(df, index_cols):
    pivot_df = df.pivot_table(index=index_cols, columns='ORDER YEAR-Mth', values='ORDER QTY CONVERTED', aggfunc="sum").reset_index()
    return pivot_df

# Use function to create pivoted dataframes for both DC and Account level roll-ups
df_ATS = create_pivot(df_usage_converted_YesATS, ['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU', 'Purchase UOM', '$ PUOM Unit Cost', 'OM Item Description'])
df_DClevelusagerules = create_pivot(df_usage_converted_YesATS, ['LOC', f'{scorecard_date} OM SKU', 'Purchase UOM'])

# Create a list of header names for ORDER QTY column(s) to call on them in below functions
date_columns_list = df_ATS.columns[6:]

### Create dataframe to capture MMIS IDs and line counts
# Step 1: Count occurrences for each MMIS ID
df_MMISIDs = (
    df_usage_converted_YesATS
    .groupby(['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU', 'MMIS ID'])
    .size()
    .reset_index(name='Count')  # Store the counts in a new column
)

# Step 2: Sum counts regardless of MMIS ID for each group
summed_counts = (
    df_MMISIDs.groupby(['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU'])
    .agg(Total_Count=('Count', 'sum'))  # Sum counts
    .reset_index()
)

# Step 3: Identify the MMIS ID with the highest count for each group
highest_mmis = (
    df_MMISIDs.loc[df_MMISIDs.groupby(['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU'])['Count'].idxmax()]
    .reset_index(drop=True)  # Reset index for cleanliness
)

# Step 4: Merge summed counts with the MMIS ID with the highest count
final_linecounts_MMISIDs = (
    pd.merge(summed_counts, highest_mmis[['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU', 'MMIS ID']],
             on=['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU'], how='left')
)

# Calculate the average monthly line count
# final_counts will have the structure you need, including summed counts and the MMIS ID with the highest count
final_linecounts_MMISIDs['Avg Monthly Line Count'] = (final_linecounts_MMISIDs['Total_Count'] / numberofmonths).round(1)

# Bring in line counts, MMIS IDs, and Blue Yonder metadata into main ATS dataframe
df_ATS = df_ATS.merge(final_linecounts_MMISIDs, on=['LOC', 'OM ACCOUNT', f'{scorecard_date} OM SKU'], how='left')
df_ATS = pd.merge(df_ATS, df_by, on=['LOC', f'{scorecard_date} OM SKU'], how='left')

# PO usage format: Calculate statistics around usage trends
def calculate_POstatistics(df, date_columns_list):
    df['Count Last 6mths w/ Activity'] = df[date_columns_list[-6:]].count(axis=1)
    df['Avg Order Qty Last 6mth'] = df[date_columns_list[-6:]].mean(axis=1)
    df['Avg Order Qty Last 3mth'] = df[date_columns_list[-3:]].mean(axis=1)
    df['Avg Order Qty Last 6mth 75%'] = df['Avg Order Qty Last 6mth'] * 0.75
    df['Sum Order Qty Last 6mth'] = df[date_columns_list[-6:]].sum(axis=1)
    df['Standard Deviation Last 6mth'] = df[date_columns_list[-6:]].std(axis=1)
    df['Variation in Usage Trend (CoV)'] = df['Standard Deviation Last 6mth'] / df['Avg Order Qty Last 6mth']
    df['Activity Last 6mth?'] = np.where(df[date_columns_list[-6:]].sum(axis=1) > 0, 'Y', 'N')
    df['Activity Last 4mth?'] = np.where(df[date_columns_list[-4:]].sum(axis=1) > 0, 'Y', 'N')
    df['Activity Last 3mth?'] = np.where(df[date_columns_list[-3:]].sum(axis=1) > 0, 'Y', 'N')
    df['Sum Qty First 3mth'] = df[date_columns_list[-6:-3]].sum(axis=1)
    df['Sum Qty Last 3mth'] = df[date_columns_list[-3:]].sum(axis=1)
    df['Usage Trend'] = np.where(df['Sum Qty Last 3mth'] > df['Sum Qty First 3mth'], 'Up', 'Down')
    return df

# PO usage format: ATS rules determinations
def dc_level_POusagerules(row):
    if row['Activity Last 6mth?'] == 'N':
        return 'Rule 1a'
    if row['Activity Last 4mth?'] == 'N':
        return 'Rule 1b'
    if row['Activity Last 3mth?'] == 'N':
        return 'Rule 1c'
    if row['Count Last 6mths w/ Activity'] >= 4:
        return 'Rule 3a' if row['Usage Trend'] == 'Up' else 'Rule 3b'
    if row['Count Last 6mths w/ Activity'] in [2, 3]:
        return 'Rule 4a'
    if row['Count Last 6mths w/ Activity'] == 1:
        return 'Rule 5a'

# Monthly usage format: ATS rules determination
def dc_level_mthlyusagerules(row, date_columns_list):
    total_sum = row[date_columns_list].sum()/(numberofmonths)
    if total_sum >= 0.25:
        return 'Y ATS: Stock item'
    if total_sum == 0:
        return 'N ATS: Sell as nonstock, no usage'
    if total_sum < 0.25:
        return 'N ATS: Sell as nonstock, below stocking criteria'

# PO usage format: ATS qty calculation logic
def calculate_ATS_qty_PO(row, date_columns_list):
    if row['ATS DC Level Usage Rule'] == 'Rule 1a' or row['ATS DC Level Usage Rule'] == 'Rule 1b':
        return 0
    elif row['ATS DC Level Usage Rule'] == 'Rule 1c':
        value = row[date_columns_list].sum() / 6
        return (round(value, 1) if value > 2 else round(value, 2))
    elif row['ATS DC Level Usage Rule'] == 'Rule 3a':
        value = row['Avg Order Qty Last 3mth'] if row['Sum Qty Last 3mth'] > 0 else row['Avg Order Qty Last 6mth']
        return (math.floor(value) if value > 2 else round(value, 2))
    elif row['ATS DC Level Usage Rule'] == 'Rule 3b':
        value = row['Avg Order Qty Last 6mth']
        return (math.floor(value) if value > 2 else round(value, 2))
    elif row['ATS DC Level Usage Rule'] == 'Rule 4a':
        value = row['Avg Order Qty Last 6mth 75%']
        return (math.floor(value) if value > 2 else round(value, 2))
    elif row['ATS DC Level Usage Rule'] == 'Rule 5a':
        value = row['Sum Order Qty Last 6mth'] / 6
        return (round(value, 1) if value > 2 else round(value, 2))
    return np.nan

# Monthly usage format: ATS qty calculation logic (Simple average)
def calculate_ATS_qty_Mthly(df, date_columns_list):
    df['QTY ATS TO LOAD @ PUOM'] = df[date_columns_list].sum(axis=1)/(numberofmonths)
    return df

# Main ATS determination and calculation processed
if usageformat == 'PO':
    # 1. Calculate statistics for both DC and Account level
    df_DClevelusagerules = calculate_POstatistics(df_DClevelusagerules, date_columns_list)
    df_ATS = calculate_POstatistics(df_ATS, date_columns_list)

    # 2. Determine usage rules for ATS qty calculations at DC level
    df_DClevelusagerules['ATS DC Level Usage Rule'] = df_DClevelusagerules.apply(dc_level_POusagerules, axis=1)

    # 3. Merge DC-level rules into account-level data
    df_ATS = df_ATS.merge(df_DClevelusagerules[['LOC', f'{scorecard_date} OM SKU', 'ATS DC Level Usage Rule']],
                          on=['LOC', f'{scorecard_date} OM SKU'], how='left')

    # 4. Calculate ATS qtys for PO data
    df_ATS['QTY ATS TO LOAD @ PUOM'] = df_ATS.apply(lambda row: calculate_ATS_qty_PO(row, date_columns_list), axis=1)

    # 5. Create column ATS Build Category that buckets based on ATS determinations
    df_ATS['ATS Build Category'] = np.select(
        [df_ATS['Activity Last 6mth?'] == 'N',
        df_ATS['Activity Last 4mth?'] == 'N',
        df_ATS['QTY ATS TO LOAD @ PUOM'] < 0.25,
        df_ATS['QTY ATS TO LOAD @ PUOM'] >= 0.25],
        ['N ATS: No usage last 4+ mths',
        'N ATS: No usage last 4+ mths',
        'N ATS: usage below stocking criteria',
        'Y ATS: Stock Item'],
        default=''
    )

elif usageformat == 'monthly':
    # 1. Determine ATS status at DC level based on summed monthly data
    df_DClevelusagerules['ATS Build Category'] = df_DClevelusagerules.apply(lambda row: dc_level_mthlyusagerules(row, date_columns_list), axis=1)

    # 2. Merge DC-level usage rules into account-level data
    df_ATS = df_ATS.merge(df_DClevelusagerules[['LOC', f'{scorecard_date} OM SKU', 'ATS Build Category']],
                          on=['LOC', f'{scorecard_date} OM SKU'], how='left')

    # 3. Calculate ATS qtys for monthly data (simple sum)
    df_ATS = calculate_ATS_qty_Mthly(df_ATS, date_columns_list)

# Calculate estimated ATS Costs using PUOM Unit Costs
df_ATS['$ Estimated ATS'] = df_ATS['QTY ATS TO LOAD @ PUOM'] * df_ATS['$ PUOM Unit Cost']
df_ATS = df_ATS.sort_values(by='$ Estimated ATS', ascending=False)

# Function to fill any N ATS records with the initial ATS load
def initial_atsscorecardbuildcategory(row):
    if pd.isnull(row['ATS Build Category']):
        if row['Conversion Error Note'] == 'NO UOM Match':
            return 'N ATS: UOM Conversion Error'
        elif row[f'{scorecard_date} READY RELEASE ATS'] != 'YES':
            return 'N ATS: N ready release for ATS'
    return row['ATS Build Category']

if ATS_workflow_type == 'initial':
    # Merge ATS Build Category data from df_ATS
    df_usage_converted = df_usage_converted.merge(
        df_ATS[['OM ACCOUNT', f'{scorecard_date} OM SKU', 'ATS Build Category']],
        on=['OM ACCOUNT', f'{scorecard_date} OM SKU'],
        how='left'
    )

    # Fill ats Build Category for N ready release items
    df_usage_converted['ATS Build Category'] = df_usage_converted.apply(initial_atsscorecardbuildcategory, axis=1)

elif ATS_workflow_type == 'update':
    # Merge to get new ATS Build Category data from df_ATS
    df_usage_converted = df_usage_converted.merge(
        df_ATS[['OM ACCOUNT', f'{scorecard_date} OM SKU', 'ATS Build Category']],
        on=['OM ACCOUNT', f'{scorecard_date} OM SKU'], 
        how='left',
        suffixes=('', '_new')  # Adding suffix to avoid overwriting
    )
    
    # Update ATS Build Category only where df_ATS has a non-null value
    df_usage_converted['ATS Build Category'] = np.where(
        # First condition: use new ATS Build Category if available
        df_usage_converted['ATS Build Category_new'].notna(),
        df_usage_converted['ATS Build Category_new'],
        # Second condition: if last week's READY RELEASE ATS is 'YES' and current is 'NO'
        np.where(
            (df_usage_converted[f'{scorecard_date_prev} READY RELEASE ATS'] == 'YES') &
            (df_usage_converted[f'{scorecard_date} READY RELEASE ATS'] == 'NO'),
            'N ATS: N ready release for ATS',
            # Default to existing ATS Build Category if neither condition is met
            df_usage_converted['ATS Build Category']
        )
    )

    # Drop the temporary column used for merging
    df_usage_converted.drop(columns=['ATS Build Category_new'], inplace=True)

# Reorder columns
df_ATS = df_ATS.reindex(columns=['LOC', f'{scorecard_date} OM SKU', 'QTY ATS TO LOAD @ PUOM', 'Purchase UOM', 'OM ACCOUNT'
                                 , 'MMIS ID', 'Avg Monthly Line Count', 'ATS Build Category', '$ Estimated ATS'
                                 , '$ PUOM Unit Cost', 'OM Item Description', 'ATS DC Level Usage Rule', 'DC 4wk fcst', 'Days Lead Time', 'IFLAG'
                                 , *date_columns_list # This code unpacks the list of date columns
                                 , 'Count Last 6mths w/ Activity', 'Avg Order Qty Last 6mth', 'Avg Order Qty Last 3mth', 'Avg Order Qty Last 6mth 75%', 'Sum Order Qty Last 6mth'
                                 , 'Standard Deviation Last 6mth', 'Variation in Usage Trend (CoV)', 'Activity Last 6mth?', 'Activity Last 4mth?', 'Activity Last 3mth?'])

df_usage_converted = df_usage_converted.reindex(columns=[*rawusagecolumns[:-2] # This removes ATS Build Category from the reindex so it isn't brought in twice
                                                        , *(['LOC', 'OM ACCOUNT'] if ATS_workflow_type == 'initial' else [])
                                                        , f'{scorecard_date} READY RELEASE ATS'
                                                        , f'{scorecard_date} ADD/CHANGE DATE (ATS)', f'{scorecard_date} SCORECARD CATEGORY'
                                                        , f'{scorecard_date} OM SKU'
                                                        , 'PXR Update Category'
                                                        , 'ATS Build Category', 'Conversion Error Note'
                                                        , 'ORDER QTY CONVERTED', 'OM Item Description', '$ PUOM Unit Cost'
                                                        , 'Base UOM', 'Base UOM Factor'
                                                        , 'Purchase UOM', 'Purchase UOM Factor', 'Alt UOM 1', 'Alt UOM 1 Factor'
                                                        , 'Alt UOM 2', 'Alt UOM 2 Factor', 'Alt UOM 3', 'Alt UOM 3 Factor'
                                                        , 'Alt UOM 4', 'Alt UOM 4 Factor', 'Alt UOM 5', 'Alt UOM 5 Factor'
                                                        , 'Alt UOM 6', 'Alt UOM 6 Factor'])

In [7]:
# Create Excel writer instance
file_path = f"{projectoutputs_folder}/{customer_name}_ATSFile_{ATS_workflow_type}_{scorecard_date}.xlsx"

with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    # Dictionary mapping of sheet names to dataframes
    sheets = {
        'ATS Build': df_ATS,
        'Raw Usage': df_usage_converted,
        'UOM Conversion Errors': df_conversionerror,
         **({'ATS Removes': df_updates_removes} if ATS_workflow_type == 'update' else {})
    }

    # Write dataframes to their respective sheets in a single pass
    for sheet_name, df in sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Get the xlsxwriter workbook object and define formats
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'center',
        'align': 'center',
        'bg_color': '#D9EAD3'
    })
    currency_format = workbook.add_format({'num_format': '$#,##0.00'})

    # Function to format a worksheet based on the dataframe columns
    def format_worksheet(worksheet, df):
        if df.empty:
            return  # Exit if the DataFrame is empty, do not format

        for col_num, value in enumerate(df.columns):
            worksheet.write(0, col_num, value, header_format)  # Apply header format

            # Calculate column width only if the DataFrame has rows
            if not df[value].empty:
                column_width = max(len(str(val)) for val in df[value].values) + 2  # Dynamically set column width based on content
                worksheet.set_column(col_num, col_num, column_width)  # Adjust column width based on data content

                # Apply currency format if column header starts with $
                if value.startswith('$'):
                    worksheet.set_column(col_num, col_num, None, currency_format)

        worksheet.autofilter(0, 0, 0, len(df.columns) - 1)  # Add autofilter to header
        worksheet.freeze_panes(1, 0)  # Freeze top row

    # Apply formatting to each worksheet after data is written
    for sheet_name, df in sheets.items():
        format_worksheet(writer.sheets[sheet_name], df)


    # Apply formatting to each worksheet after data is written
    for sheet_name, df in sheets.items():
        format_worksheet(writer.sheets[sheet_name], df)